### ספריות


In [ ]:
import os
import pandas as pd
# from adding_an_addition_following_the_index import adding_an_addition
# from functions import change_Muni_Heb_to_Muni_Eng
# from division_into_traffic_zones_of_plans import division_into_traffic_zones
# from export_geo_layer_for_client_control import export_geo_layer
# from export_index_layer_for_client_control import export_index_layer
# from forecast import clientTaz
# from geographical_features import add_geographical_Features
# from index_layer import index_layer_fun
# from status_exists_for_control import export_status_exists
# from uploading_index_table_elements import uploading_index_table
# from uploading_index_table_elements import uploading_index_table
# from export_forecast_2040 import export_forecast
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [3]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

### העלת משתנים להרצת הקוד


In [ ]:
cwd = os.getcwd()

create_forecast_basic_ad_hoc_location = r'{}\create_forecast_basic_ad_hoc'.format(cwd)

df_inputs_outputs = pd.read_excel(r'inputs_outputs.xlsx')

,discription,location
0,מיקום לקוח,C:\Users\dpere\Documents\JTMT\Projects\תחזיות_...
1,שם תרחיש (forecast_version),realy
2,v_date,240818_with_poten
3,מיקום פלט תחזית בסיס לפי גירסא,C:\Users\dpere\Documents\JTMT\forecast_by_vers...
4,מספר גירסא,1


In [ ]:
# מיקום לקוח
client_data_folder_location = df_inputs_outputs['location'][0]

# שם תרחיש (forecast_version)
forecast_version = df_inputs_outputs['location'][1]

# v_date
v_date = df_inputs_outputs['location'][2]

# מיקום פלט תחזית בסיס לפי גירסא
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]

# מספר גירסא
forecast_version_number = df_inputs_outputs['location'][1]

index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)